## 🛠️ Implementing CycloneDDS with Iceoryx (Shared Memory)

Implementing **Eclipse CycloneDDS** with **Iceoryx** for shared memory communication involves three main components: building the libraries, configuring both middlewares, and running the applications with the necessary environment settings. This approach enables **zero-copy** data transfer for high-performance intra-process and inter-process communication on the same machine.

-----

### 1\. Prerequisites and Building

First, you need to build both Iceoryx and a specific branch of CycloneDDS that includes Iceoryx support. The following are typical build steps (assuming you are building in your home directory, `~/`):

1.  **Install prerequisites** (e.g., `cmake`, `libacl1-dev`, `libncurses5-dev`, `pkg-config`, `maven`).
2.  **Build and Install Iceoryx (RouDi):**
    ```bash
    git clone https://github.com/eclipse-iceoryx/iceoryx.git -b release_2.0
    cd iceoryx
    cmake -Bbuild -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=install -DROUDI_ENVIRONMENT=on -DBUILD_SHARED_LIBS=ON -Hiceoryx_meta
    cmake --build build --config Release --target install
    ```
3.  **Build and Install CycloneDDS with Iceoryx support:**
    ```bash
    git clone https://github.com/eclipse-cyclonedds/cyclonedds.git
    cd cyclonedds
    # Check out a branch that supports iceoryx if master is not the correct one,
    # or use -DENABLE_ICEORYX=On if available in your version.
    cmake -Bbuild -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=install -DENABLE_ICEORYX=On -DBUILD_EXAMPLES=On -DCMAKE_PREFIX_PATH=~/iceoryx/install/
    cmake --build build --config Release --target install
    ```

-----

### 2\. Configuration Files

You need separate configuration files for both Iceoryx and CycloneDDS.

#### 📝 Iceoryx Configuration (`iox_config.toml`)

This file defines the shared memory pool segments. The `size` should be 64 bytes larger than your largest message type (due to header information), and `count` is the number of blocks available. For example:

```toml
[general]
version = 1
[[segment]]
    [[segment.mempool]]
    # Example: block size for a 16384 byte message (16384 + 64 bytes)
    size = 16448
    count = 32768
```

#### 📝 CycloneDDS Configuration (`cyclonedds.xml`)

This file explicitly enables the shared memory exchange layer via Iceoryx.

```xml
<?xml version="1.0" encoding="UTF-8"?>
<CycloneDDS xmlns="https://cdds.io/config">
    <Domain id="any">
        <SharedMemory>
            <Enable>true</Enable>
            <LogLevel>info</LogLevel>
        </SharedMemory>
    </Domain>
</CycloneDDS>
```

-----

### 3\. Execution Steps

To run your CycloneDDS applications with shared memory, you need three separate terminal sessions:

#### A. Start RouDi (Routing and Discovery Daemon)

Iceoryx requires its daemon to manage shared memory segments and facilitate discovery.

```bash
# In Terminal 1
~/iceoryx/build/iox-roudi -c iox_config.toml
```

#### B. Run Publisher and Subscriber

In the remaining terminals, run your DDS applications (e.g., the built-in `ShmThroughputPublisher` and `ShmThroughputSubscriber` examples) after setting the required environment variables:

```bash
# In Terminal 2 (Publisher)
export LD_LIBRARY_PATH=~/iceoryx/install/lib:$LD_LIBRARY_PATH
export CYCLONEDDS_URI=file://cyclonedds.xml
~/cyclonedds/build/bin/ShmThroughputPublisher 16384 0 1 10 "Throughput example"

# In Terminal 3 (Subscriber)
export LD_LIBRARY_PATH=~/iceoryx/install/lib:$LD_LIBRARY_PATH
export CYCLONEDDS_URI=file://cyclonedds.xml
~/cyclonedds/build/bin/ShmThroughputSubscriber 10 0 "Throughput example" 16384
```

> **Key Considerations for Zero-Copy:**
>
>   * The DDS data type should be **fixed-size** (no strings, sequences, or other dynamic containers) to enable true zero-copy.
>   * For maximum performance, use the **Loan Mechanism** in your DDS application code.

You can watch [this webinar](https://www.youtube.com/watch?v=5GpROveP6Hg) on YouTube to learn more about how shared memory exchange works in Cyclone DDS.
http://googleusercontent.com/youtube_content/1